In [1]:
import numpy as np
import cv2
from PIL import Image
from matplotlib import pyplot as plt
%matplotlib inline
import os
from os import listdir
from os.path import isfile, join
import pandas as pd

In [2]:

df_train=pd.read_csv('main_final.csv',index_col=False)
labels=df_train[['784']]

In [3]:
df_train.drop(df_train.columns[[784]],axis=1,inplace=True)
print(df_train.head())
print(labels.shape)

     0    1    2    3    4    5    6    7    8    9  ...  774  775  776  777  \
0    0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
1  255  255  255  255  255  255  255  255  255  255  ...    0    0    0    0   
2    0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
3    0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
4    0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   

   778  779  780  781  782  783  
0    0    0    0    0    0    0  
1    0    0    0    0    0    0  
2    0    0    0    0    0    0  
3    0    0    0    0    0    0  
4    0    0    0    0    0    0  

[5 rows x 784 columns]
(61512, 1)


In [4]:
np.random.seed(1212)
import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers
from keras.layers import Input, Dense
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K


Using TensorFlow backend.


In [5]:
K.set_image_data_format('channels_first')

In [6]:
labels=np.array(labels)

In [7]:
from keras.utils.np_utils import to_categorical
cat=to_categorical(labels,num_classes=21)

In [8]:
print(cat[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [9]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
l=[]
for i in range(61512):
    l.append(np.array(df_train[i:i+1]).reshape(1,28,28))

In [11]:
np.random.seed(7)

In [12]:
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback

In [13]:
def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

In [14]:
tfback._get_available_gpus = _get_available_gpus
tfback._get_available_gpus()

[]

In [19]:
model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape=(1 , 28, 28), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(21, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
from keras.models import model_from_json

In [21]:
model.fit(np.array(l), cat, epochs=10, batch_size=200,shuffle=True,verbose=1)

Epoch 1/10
61512/61512 [==============================] - 29s 465us/step - loss: 1.5752 - accuracy: 0.6322
Epoch 2/10
61512/61512 [==============================] - 31s 499us/step - loss: 0.3751 - accuracy: 0.8901
Epoch 3/10
61512/61512 [==============================] - 33s 530us/step - loss: 0.2336 - accuracy: 0.9297
Epoch 4/10
61512/61512 [==============================] - 35s 564us/step - loss: 0.1712 - accuracy: 0.9492
Epoch 5/10
61512/61512 [==============================] - 35s 562us/step - loss: 0.1318 - accuracy: 0.9600
Epoch 6/10
61512/61512 [==============================] - 31s 509us/step - loss: 0.1133 - accuracy: 0.9650s - loss: 0.1133 - accuracy: 0.96
Epoch 7/10
61512/61512 [==============================] - 29s 467us/step - loss: 0.0985 - accuracy: 0.9686
Epoch 8/10
61512/61512 [==============================] - 28s 456us/step - loss: 0.0861 - accuracy: 0.9726
Epoch 9/10
61512/61512 [==============================] - 29s 467us/step - loss: 0.0783 - accuracy: 0.9748
Epoc

In [22]:
model_json = model.to_json()
with open("model_final.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_final.h5")